In [ ]:
SELECT cities.name AS city, countries.name AS country, countries.region AS region FROM cities INNER JOIN countries ON cities.country_code = countries.code;

In [ ]:
SELECT c.code, name, region, e.year, fertility_rate, unemployment_rate
FROM countries AS c
INNER JOIN populations AS p
ON c.code = p.country_code
INNER JOIN economies AS e
ON c.code = e.code AND e.year = p.year;